In [2]:
from __future__ import print_function
import import_ipynb
from fenics import *
from dolfin import *
import gym
import numpy as np

class advec_diff_DVortex:
    
    def __init__(self,
                 f,
                 dt,
                 l1_upperbound,
                 l1_lowerbound,
                 u0_expr = Expression('0.0', degree=2),
                 region = None,
                 K = 1*0.022,
                 lm = 5*0.0017,
                 lx = 8.0,
                 ly = 4.0,
                 nx = 80,
                 ny = 40):
        
        """
        Parameters
        ----------
        f: C-srtring
            expression for source term
        w_expr: C-srtring
            expression for velocity field
        dt: float
            time increment
        u0_expr: C-srtring
            expression for initial concentration field (default: 0.0)
        K: float, optional
            diffusion coeff. (default 0.022)
        lm: float, optional
            coeff. for effective removal of particles by HVAC 
            (default 0.0017)
        lx: float, optional
            room length (default 8.0)
        ly: float, optional
            room width (default 8.0)
        nx: int, optional
            number of mesh cells in the length (default 32)
        ny: int, optional
            number of mesh cells in the width (default 32)
        """

        #         Instantiating the environment model with the parameters:

        self.f = f
#         self.w_expr = w_expr
        self.u0_expr = u0_expr
        self.dt = Constant(dt)
        self.l1up = l1_upperbound
        self.l1low = l1_lowerbound
        self.K = Constant(K)
        if isinstance(lm, (int, float)):
            self.lm = Constant(lm)
        else:
            self.lm = lm

        self.lx = lx
        self.ly = ly
        
        self.mesh = RectangleMesh(Point(0.0, 0.0), Point(lx, ly), nx, ny, diagonal='right')
        
        # creating the integration subdomain
        # cf = CellFunction('size_t', mesh, 0)
        if region == None:
            region = AutoSubDomain(lambda x, on: x[0] <= lx and x[0]>0.5*lx and x[1] <= ly and x[1]>0.0)
        
        cf = MeshFunction("size_t", self.mesh, self.mesh.topology().dim(), 0)
        region.mark(cf, 1)
        self.dx_sub = Measure('dx', subdomain_data=cf)

    def reset(self):
        """
        defines function spaces for velocity field and concentration.
        defines trial and test functions.
        defines subdomain for integration purposes.
        sets initial value for concentration
        """
        
        # Define function space for velocity
#         W = VectorFunctionSpace(self.mesh, 'P', 2)
        
        # Define function space for concentration
        P1 = FiniteElement('P', triangle, 1)
        self.V = FunctionSpace(self.mesh, P1)
        self.W_funcspace = VectorFunctionSpace(self.mesh, 'P', 2)

        
        # Define initial value
        u_n = interpolate(self.u0_expr, self.V)
        self.state = u_n
        return self.state
    
    def step(self, action):
        """
        takes input action (velocity field w) and simulates the system 
        for one time step (dt)
        
        this function returns the new system state, the total reward and
        time spent during that step as well as time series of temp,
        reward, and time spent in that step. It also returns a variable
        named "done" which indicates the end of total simulation if its
        value is set to True.
        """

        u_n = self.state
        
        
        l1 = self.denormalize_action_func(action)
        
        wx_l = 1
        wx_r = 1
        wy_l = wx_l*self.ly/l1
        wy_r = wx_r*self.ly/(self.lx-l1)
        w_expr = Expression(('x[0]<l1 ? wx_l*sin(pi*1/l1*x[0])*cos(cy*pi*x[1]) : \
                              wx_r*sin(pi*1/(lx-l1)*(x[0]-l1))*cos(cy*pi*x[1])',
                             'x[0]<l1 ? -wy_l*cos(pi*1/l1*x[0])*sin(cy*pi*x[1]): \
                             -wy_r*cos(pi*1/(lx-l1)*(x[0]-l1))*sin(cy*pi*x[1])'),
                            wx_l=wx_l, wx_r=wx_r, wy_l=wy_l, wy_r=wy_r, l1=l1, lx=self.lx, cy=1/self.ly, degree=2)

#         w_expr = Expression(('x[1]<ly/2 ? wx : -wx', 'wy'), degree=2, wx=0.550, wy=0.0, ly=self.ly)

        w = interpolate(w_expr, self.W_funcspace)
        
#         w = action
        
        # Define test and trial functions
        u = TrialFunction(self.V)
        v = TestFunction(self.V)
        
        # Define variational problem
        F = ((u - u_n) / self.dt)*v*dx + dot(w, grad(u))*v*dx \
        + self.K*dot(grad(u), grad(v))*dx + self.lm*u*v*dx  \
        - self.f*v*dx
        a, L = lhs(F), rhs(F)
        
        # Time-stepping
        u = Function(self.V)
        
        # Compute solution
        solve(a == L, u)
        
        # calculate the immediate reward
        reward = assemble(u_n*self.dx_sub(1)) - assemble(u*self.dx_sub(1))
        
        # Update the state
        self.state.assign(u)
        
        # calculate the spatial integral over the subdomain dx_sub 
        integ_sp = assemble(u*self.dx_sub(1))
        
        return self.state, integ_sp, reward
    
    def denormalize_action_func(self,a):
        denormalized_a = (self.l1up-self.l1low)*(a+1)/2+self.l1low
        return denormalized_a

In [2]:
class advec_diff:
    
    def __init__(self,
                 f,
                 dt,
                 l1_upperbound,
                 l1_lowerbound,
                 u0_expr = Expression('0.0', degree=2),
                 w_expr = None,
                 mesh = None,
                 region = None,
                 K = 1*0.022,
                 lm = 5*0.0017,
                 lx = 8.0,
                 ly = 4.0,
                 nx = 80,
                 ny = 40):
        
        """
        Parameters
        ----------
        f: C-srtring
            expression for source term
        w_expr: C-srtring
            expression for velocity field
        dt: float
            time increment
        u0_expr: C-srtring
            expression for initial concentration field (default: 0.0)
        K: float, optional
            diffusion coeff. (default 0.022)
        lm: float, optional
            coeff. for effective removal of particles by HVAC 
            (default 0.0017)
        lx: float, optional
            room length (default 8.0)
        ly: float, optional
            room width (default 8.0)
        nx: int, optional
            number of mesh cells in the length (default 32)
        ny: int, optional
            number of mesh cells in the width (default 32)
        """

        #         Instantiating the environment model with the parameters:

        self.f = f
#         self.w_expr = w_expr
        self.u0_expr = u0_expr
        self.dt = Constant(dt)
        self.l1up = l1_upperbound
        self.l1low = l1_lowerbound
        self.K = Constant(K)
        if isinstance(lm, (int, float)):
            self.lm = Constant(lm)
        else:
            self.lm = lm

        self.lx = lx
        self.ly = ly
        
        if w_expr is None:
            self.w_expr = Expression(('wx', 'wy'), degree=2, wx=0.15, wy=0.0)
        else:
            self.w_expr = w_expr

        if mesh is None:
            self.mesh = RectangleMesh(Point(0.0, 0.0), Point(lx, ly), nx, ny, diagonal='right')
        else:
            self.mesh = mesh
        
        # creating the integration subdomain
        # cf = CellFunction('size_t', mesh, 0)
        if region == None:
            region = AutoSubDomain(lambda x, on: x[0] <= lx and x[0]>0.5*lx and x[1] <= ly and x[1]>0.0)
        
        cf = MeshFunction("size_t", self.mesh, self.mesh.topology().dim(), 0)
        region.mark(cf, 1)
        self.dx_sub = Measure('dx', subdomain_data=cf)

    def reset(self):
        """
        defines function spaces for velocity field and concentration.
        defines trial and test functions.
        defines subdomain for integration purposes.
        sets initial value for concentration
        """

        
        # Define function space for concentration and velocity field
        P1 = FiniteElement('P', triangle, 1)
        self.V = FunctionSpace(self.mesh, P1)
        self.W = VectorFunctionSpace(self.mesh, 'P', 2)

        # define velocity field
        w = interpolate(self.w_expr, self.W)
        
        # Define initial value
        u_n = interpolate(self.u0_expr, self.V)
        self.state = u_n
        
        # Define test and trial functions
        u = TrialFunction(self.V)
        v = TestFunction(self.V)
        
        # Define variational problem
        F = ((u - u_n) / self.dt)*v*dx + dot(w, grad(u))*v*dx \
        + self.K*dot(grad(u), grad(v))*dx + self.lm*u*v*dx  \
        - self.f*v*dx
        self.a, self.L = lhs(F), rhs(F)
        
        # defining function u before time-stepping
        self.u = Function(self.V)
        
        return self.state
    
    def step(self):
        """
        takes input action (velocity field w) and simulates the system 
        for one time step (dt)
        
        this function returns the new system state, the total reward and
        time spent during that step as well as time series of temp,
        reward, and time spent in that step. It also returns a variable
        named "done" which indicates the end of total simulation if its
        value is set to True.
        """

        u_n = self.state
#         w = self.w
                
        u = self.u
        
        # Compute solution
        solve(self.a == self.L, u)
        
        # calculate the immediate reward
        reward = assemble(u_n*self.dx_sub(1)) - assemble(u*self.dx_sub(1))
        
        # Update the state
        self.state.assign(u)
        
        # calculate the spatial integral over the subdomain dx_sub 
        integ_sp = assemble(u*self.dx_sub(1))
        
        return self.state, integ_sp, reward
    
    def denormalize_action_func(self,a):
        denormalized_a = (self.l1up-self.l1low)*(a+1)/2+self.l1low
        return denormalized_a

In [4]:
from __future__ import print_function
import import_ipynb
from fenics import *
from dolfin import *
import gym
import numpy as np

class advec_diff_hp:
    
    def __init__(self,
                 f1,
                 dt,
                 w_expr,
                 xhp_upperbound,
                 xhp_lowerbound,
                 yhp = 3.0,
                 R2 = 2.5,
                 eps2 = 0.1,
                 f2_expr = None,
                 u0_expr = Expression(('0.0','0.0'), degree=2),
                 region = None,
                 K1 = 1*0.022,
                 K2 = 1*0.022,
                 lm1 = 5*0.0017,
                 lm2 = 5*0.0017,
                 alpha1 = 1.0,
                 alpha2 = 1.0,
                 lx = 8.0,
                 ly = 4.0,
                 nx = 80,
                 ny = 40):
        
        """
        Parameters
        ----------
        f1: C-srtring
            expression for COVID source term
        w_expr: C-srtring
            expression for velocity field
        dt: float
            time increment
        xhp_upperbound: float
            upperbound of xhp
        xhp_lowerbound: float
            lowerbound of xhp
        yhp: float, optional
            y coordinate of HP source (default = 3.0)
        Dx: float, optional
            width of source area for both COVID and HP (default = 0.25)
        Dy: float, optional
            height of source area for both COVID and HP (default = 0.25)
        R2: float, optional
            HP source coeff. (default= 2.5)
        region: Dolfin object, optional
            spatial region of interest for COVID minimization (default: the whole room)
        u0_expr: C-srtring
            expression for initial concentration fields for COVID and Hydrogen Peroxide (default: 0.0, 0.0)
        K1: float, optional
            diffusion coeff. for COVID (default 0.022)
        K2: float, optional
            diffusion coeff. for HP (default 0.022)
        lm1: float, optional
            coeff. for effective removal of COVID particles by HVAC
            (default 5* 0.0017)
        lm2: float, optional
            coeff. for effective removal of HP particles by HVAC
            (default 5* 0.0017)
        alpha1: float, optional
            reaction coeff. between COVID and HP in add. diff. equation for COVID
        alpha2: float, optional
            reaction coeff. between COVID and HP in add. diff. equation for HP
        lx: float, optional
        lx: float, optional
            room length (default 8.0)
        ly: float, optional
            room width (default 8.0)
        nx: int, optional
            number of mesh cells in the length (default 32)
        ny: int, optional
            number of mesh cells in the width (default 32)
        """

        #         Instantiating the environment model with the parameters:

        self.f1 = f1
        self.w_expr = w_expr
        self.u0_expr = u0_expr
        self.dt = Constant(dt)
        self.xhpup = xhp_upperbound
        self.xhplow = xhp_lowerbound
        self.yhp = yhp
        self.R2 = R2
        self.eps2 = eps2
        self.K1 = Constant(K1)
        self.K2 = Constant(K2)
        if isinstance(lm1, (int, float)):
            self.lm1 = Constant(lm1)
        else:
            self.lm1 = lm1
        if isinstance(lm2, (int, float)):
            self.lm2 = Constant(lm2)
        else:
            self.lm2 = lm2

        self.alpha1 = Constant(alpha1)
        self.alpha2 = Constant(alpha2)
        
        self.lx = lx
        self.ly = ly
        
        if f2_expr is None:
            self.f2_expr = 'R/(pi*pow(eps,1))*exp(-(pow(x[0]-xs,2)+pow(x[1]-ys,2))/pow(eps,1))'
        else:
            self.f2_expr = f2_expr
        
        self.mesh = RectangleMesh(Point(0.0, 0.0), Point(lx, ly), nx, ny, diagonal='right')
        
        # creating the integration subdomain
        # cf = CellFunction('size_t', mesh, 0)
        if region == None:
            region = AutoSubDomain(lambda x, on: x[0] <= lx and x[0]>0.0*lx and x[1] <= ly and x[1]>0.0)
        
        cf = MeshFunction("size_t", self.mesh, self.mesh.topology().dim(), 0)
        region.mark(cf, 1)
        self.dx_sub = Measure('dx', subdomain_data=cf)

    def reset(self):
        """
        defines function spaces for velocity field and concentration.
        defines trial and test functions.
        defines subdomain for integration purposes.
        sets initial value for concentration
        """
        
        # Define function space for velocity
#         W = VectorFunctionSpace(self.mesh, 'P', 2)
        
        # Define function space for concentrations
        P1 = FiniteElement('P', triangle, 1)
        element = MixedElement([P1, P1])
        self.V = FunctionSpace(self.mesh, element)
        self.W = VectorFunctionSpace(self.mesh, 'P', 2)

        # Define test functions
        self.v1, self.v2 = TestFunctions(self.V)

        # Define functions for velocity and concentrations
        w = Function(self.W)
        self.u = Function(self.V)
        u_n = Function(self.V)
        
        # define velocity field
        self.w = interpolate(self.w_expr, self.W)

        # Define initial value

        u_n = interpolate(self.u0_expr, self.V)
        self.state = u_n
        
        return self.state
    
    def step(self, action):
        """
        takes input action (xhp) and simulates the system 
        for one time step (dt)
        
        this function returns the new system state, the spatial integraion 
        of COVID concentration in the region of interest, and the immediate reward
        """

        u_n = self.state
        u_n1, u_n2 = split(u_n)
        
        u = self.u
        u1, u2 = split(u)
        
        v1 = self.v1
        v2 = self.v2

        xhp = self.denormalize_action_func(action)

        f2 = Expression(self.f2_expr, degree=4, xs=xhp, ys=self.yhp, eps=self.eps2, R=self.R2)

        # Define variational problem
        F = ((u1 - u_n1) / self.dt)*v1*dx + dot(self.w, grad(u1))*v1*dx \
        + self.K1*dot(grad(u1), grad(v1))*dx + self.lm1*u1*v1*dx - self.f1*v1*dx + self.alpha1*u1*u2*v1*dx\
        +((u2 - u_n2) / self.dt)*v2*dx + dot(self.w, grad(u2))*v2*dx \
        + self.K2*dot(grad(u2), grad(v2))*dx + self.lm2*u2*v2*dx - f2*v2*dx + self.alpha2*u1*u2*v2*dx\
        
        # Solve variational problem for time step
        solve(F == 0, u)
        
        _u1, _u2 = u.split()
        _u_n1, _u_n2= u_n.split()

        # calculate the immediate reward
        reward = assemble(_u_n1*self.dx_sub(1)) - assemble(_u1*self.dx_sub(1))

        # calculate the spatial integral over the subdomain self.dx_sub 
        integ_sp_COVID = assemble(_u1*self.dx_sub(1))
        integ_sp_HP = assemble(_u2*self.dx_sub(1))
        
        # Update the state
        self.state.assign(u)
        
        return self.state, integ_sp_COVID, integ_sp_HP, reward
    
    def denormalize_action_func(self,a):
        denormalized_a = (self.xhpup-self.xhplow)*(a+1)/2+self.xhplow
        return denormalized_a

In [3]:

class advec_diff_gym(gym.Env):
    
    def __init__(self,
                 f,
                 dt,
                 l1_upperbound,
                 l1_lowerbound,
                 T,
                 u0_expr = Expression('0.0', degree=2),
                 region = None,
                 K = 1*0.022,
                 lm = 5*0.0017,
                 lx = 8.0,
                 ly = 4.0,
                 nx = 64,
                 ny = 32):
        
        """
        Parameters
        ----------
        f: C-srtring
            expression for source term
        w_expr: C-srtring
            expression for velocity field
        dt: float
            time increment
        T: float
            final simulation time
        u0_expr: C-srtring
            expression for initial concentration field (default: 0.0)
        K: float, optional
            diffusion coeff. (default 0.022)
        lm: float, optional
            coeff. for effective removal of particles by HVAC 
            (default 0.0017)
        lx: float, optional
            room length (default 8.0)
        ly: float, optional
            room width (default 8.0)
        nx: int, optional
            number of mesh cells in the length (default 32)
        ny: int, optional
            number of mesh cells in the width (default 32)
        """

        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = gym.spaces.Discrete(1)
        
        #         Instantiating the environment model with the parameters:

        self.f = f
#         self.w_expr = w_expr
        self.u0_expr = u0_expr
        self.dt = Constant(dt)
        self.Dt = dt
        self.l1up = l1_upperbound
        self.l1low = l1_lowerbound
        self.T = T
        self.K = Constant(K)
        self.lm = Constant(lm)
        self.lx = lx
        self.ly = ly
        
        self.mesh = RectangleMesh(Point(0.0, 0.0), Point(lx, ly), nx, ny, diagonal='right')
        
        # creating the integration subdomain
        # cf = CellFunction('size_t', mesh, 0)
        if region == None:
            region = AutoSubDomain(lambda x, on: x[0] <= lx and x[0]>0.5*lx and x[1] <= ly and x[1]>0.0)
        
        cf = MeshFunction("size_t", self.mesh, self.mesh.topology().dim(), 0)
        region.mark(cf, 1)
        self.dx_sub = Measure('dx', subdomain_data=cf)

    def reset(self):
        """
        defines function spaces for velocity field and concentration.
        defines trial and test functions.
        defines subdomain for integration purposes.
        sets initial value for concentration
        """
        
        # Define function space for velocity
#         W = VectorFunctionSpace(self.mesh, 'P', 2)
        
        # Define function space for concentration
        P1 = FiniteElement('P', triangle, 1)
        self.V = FunctionSpace(self.mesh, P1)
        self.W_funcspace = VectorFunctionSpace(self.mesh, 'P', 2)

        
        # Define initial value
        u_n = interpolate(self.u0_expr, self.V)
        self.state = u_n
        self.t = 0
        obs = 0
        
        self.index=0
        
        return obs
    
    def step(self, action):
        """
        takes input action (velocity field w) and simulates the system 
        for one time step (dt)
        
        this function returns the new system state, the total reward and
        time spent during that step as well as time series of temp,
        reward, and time spent in that step. It also returns a variable
        named "done" which indicates the end of total simulation if its
        value is set to True.
        """

        u_n = self.state
        l1 = self.denormalize_action_func(action)
        
        
        wx_l = 1
        wx_r = 1
        wy_l = wx_l*self.ly/l1
        wy_r = wx_r*self.ly/(self.lx-l1)
        w_expr = Expression(('x[0]<l1 ? wx_l*sin(pi*1/l1*x[0])*cos(cy*pi*x[1]) : \
                              wx_r*sin(pi*1/(lx-l1)*(x[0]-l1))*cos(cy*pi*x[1])',
                             'x[0]<l1 ? -wy_l*cos(pi*1/l1*x[0])*sin(cy*pi*x[1]): \
                             -wy_r*cos(pi*1/(lx-l1)*(x[0]-l1))*sin(cy*pi*x[1])'),
                            wx_l=wx_l, wx_r=wx_r, wy_l=wy_l, wy_r=wy_r, l1=l1, lx=self.lx, cy=1/self.ly, degree=2)

        
        w = interpolate(w_expr, self.W_funcspace)

        # Define test and trial functions
        u = TrialFunction(self.V)
        v = TestFunction(self.V)
        
        # Define variational problem
        F = ((u - u_n) / self.dt)*v*dx + dot(w, grad(u))*v*dx \
        + self.K*dot(grad(u), grad(v))*dx + self.lm*u*v*dx  \
        - self.f*v*dx
        a, L = lhs(F), rhs(F)
        
        # Time-stepping
        u = Function(self.V)
        
        # Compute solution
        solve(a == L, u)
        
        # step up the time
        self.t += self.Dt
        
        # calculate the immediate reward
        reward = assemble(u_n*self.dx_sub(1)) - assemble(u*self.dx_sub(1))
                
        # Update the state
        self.state.assign(u)
        
        # calculate the spatial integral over the subdomain dx_sub 
        integ_sp = assemble(u*self.dx_sub(1))
        
        if self.t>=self.T:
            done = True
        else:
            done = False
        
        obs = 0
        info={}
        self.index += 1
        
        return obs, reward, done, info
    
    def denormalize_action_func(self,a):
        denormalized_a = (self.l1up-self.l1low)*(a+1)/2+self.l1low
        return denormalized_a

In [6]:
class advec_diff_hp_gym(gym.Env):
    
    def __init__(self,
                 f1,
                 dt,
                 T,
                 w_expr,
                 xhp_upperbound,
                 xhp_lowerbound,
                 yhp = 3.0,
                 R2 = 2.5,
                 eps2 = 0.1,
                 f2_expr = None,
                 u0_expr = Expression(('0.0','0.0'), degree=2),
                 region = None,
                 K1 = 1*0.022,
                 K2 = 1*0.022,
                 lm1 = 5*0.0017,
                 lm2 = 5*0.0017,
                 alpha1 = 0.01,
                 alpha2 = 0.01,
                 lx = 8.0,
                 ly = 4.0,
                 nx = 80,
                 ny = 40):
        
        """
        Parameters
        ----------
        f1: C-srtring
            expression for COVID source term
        w_expr: C-srtring
            expression for velocity field
        dt: float
            time increment
        xhp_upperbound: float
            upperbound of xhp
        xhp_lowerbound: float
            lowerbound of xhp
        yhp: float, optional
            y coordinate of HP source (default = 3.0)
        Dx: float, optional
            width of source area for both COVID and HP (default = 0.25)
        Dy: float, optional
            height of source area for both COVID and HP (default = 0.25)
        R2: float, optional
            HP source coeff. (default= 2.5)
        region: Dolfin object, optional
            spatial region of interest for COVID minimization (default: the whole room)
        u0_expr: C-srtring
            expression for initial concentration fields for COVID and Hydrogen Peroxide (default: 0.0, 0.0)
        K1: float, optional
            diffusion coeff. for COVID (default 0.022)
        K2: float, optional
            diffusion coeff. for HP (default 0.022)
        lm1: float, optional
            coeff. for effective removal of COVID particles by HVAC
            (default 5* 0.0017)
        lm2: float, optional
            coeff. for effective removal of HP particles by HVAC
            (default 5* 0.0017)
        alpha1: float, optional
            reaction coeff. between COVID and HP in add. diff. equation for COVID
        alpha2: float, optional
            reaction coeff. between COVID and HP in add. diff. equation for HP
        lx: float, optional
        lx: float, optional
            room length (default 8.0)
        ly: float, optional
            room width (default 8.0)
        nx: int, optional
            number of mesh cells in the length (default 32)
        ny: int, optional
            number of mesh cells in the width (default 32)
        """

        
        self.action_space = gym.spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = gym.spaces.Discrete(1)
        # Instantiating the environment model with the parameters:

        self.T = T
        self.f1 = f1
        self.w_expr = w_expr
        self.u0_expr = u0_expr
        self.dt = Constant(dt)
        self.Dt = dt
        self.xhpup = xhp_upperbound
        self.xhplow = xhp_lowerbound
        self.yhp = yhp
        self.R2 = R2
        self.eps2 = eps2
        self.K1 = Constant(K1)
        self.K2 = Constant(K2)
        if isinstance(lm1, (int, float)):
            self.lm1 = Constant(lm1)
        else:
            self.lm1 = lm1
        if isinstance(lm2, (int, float)):
            self.lm2 = Constant(lm2)
        else:
            self.lm2 = lm2

        self.alpha1 = Constant(alpha1)
        self.alpha2 = Constant(alpha2)
        
        self.lx = lx
        self.ly = ly
        
        if f2_expr is None:
            self.f2_expr = 'R/(pi*pow(eps,1))*exp(-(pow(x[0]-xs,2)+pow(x[1]-ys,2))/pow(eps,1))'
        else:
            self.f2_expr = f2_expr
        
        self.mesh = RectangleMesh(Point(0.0, 0.0), Point(lx, ly), nx, ny, diagonal='right')
        
        # creating the integration subdomain
        # cf = CellFunction('size_t', mesh, 0)
        if region == None:
            region = AutoSubDomain(lambda x, on: x[0] <= lx and x[0]>0.0*lx and x[1] <= ly and x[1]>0.0)
        
        cf = MeshFunction("size_t", self.mesh, self.mesh.topology().dim(), 0)
        region.mark(cf, 1)
        self.dx_sub = Measure('dx', subdomain_data=cf)

    def reset(self):
        """
        defines function spaces for velocity field and concentration.
        defines trial and test functions.
        defines subdomain for integration purposes.
        sets initial value for concentration
        """
                
        # Define function space for concentrations
        P1 = FiniteElement('P', triangle, 1)
        element = MixedElement([P1, P1])
        self.V = FunctionSpace(self.mesh, element)
        self.W = VectorFunctionSpace(self.mesh, 'P', 2)


        # Define test functions
        self.v1, self.v2 = TestFunctions(self.V)

        # Define functions for velocity and concentrations
        w = Function(self.W)
        self.u = Function(self.V)
        u_n = Function(self.V)

        # define velocity field
        self.w = interpolate(self.w_expr, self.W)

        # Define initial value
        u_n = interpolate(self.u0_expr, self.V)
        self.state = u_n
        
        self.t = 0
        obs = 0
        
        self.index=0
        
        return obs
    
    def step(self, action):
        """
        takes input action (xhp) and simulates the system 
        for one time step (dt)
        
        this function returns the new system state, the spatial integraion 
        of COVID concentration in the region of interest, and the immediate reward
        """

        u_n = self.state
        u_n1, u_n2 = split(u_n)
        
        u = self.u
        u1, u2 = split(u)
        
        v1 = self.v1
        v2 = self.v2

        xhp = self.denormalize_action_func(action)
        
        f2 = Expression(self.f2_expr, degree=4, xs=xhp, ys=self.yhp, eps=self.eps2, R=self.R2)

        # Define variational problem
        F = ((u1 - u_n1) / self.dt)*v1*dx + dot(self.w, grad(u1))*v1*dx \
        + self.K1*dot(grad(u1), grad(v1))*dx + self.lm1*u1*v1*dx - self.f1*v1*dx + self.alpha1*u1*u2*v1*dx\
        +((u2 - u_n2) / self.dt)*v2*dx + dot(self.w, grad(u2))*v2*dx \
        + self.K2*dot(grad(u2), grad(v2))*dx + self.lm2*u2*v2*dx - f2*v2*dx + self.alpha2*u1*u2*v2*dx\
        
        # Solve variational problem for time step
        solve(F == 0, u)
        
        # step up the time
        self.t += self.Dt
        
        _u1, _u2 = u.split()
        _u_n1, _u_n2= u_n.split()

        # calculate the immediate reward
        reward = assemble(_u_n1*self.dx_sub(1)) - assemble(_u1*self.dx_sub(1))

        # calculate the spatial integral over the subdomain self.dx_sub 
        integ_sp = assemble(_u1*self.dx_sub(1))
        
        # Update the state
        self.state.assign(u)
        
        if self.t>=self.T:
            done = True
        else:
            done = False
        
        obs = 0
        info={}
        self.index += 1
        
        return obs, reward, done, info
    
    def denormalize_action_func(self,a):
        denormalized_a = (self.xhpup-self.xhplow)*(a+1)/2+self.xhplow
        return denormalized_a